In [1]:
%run 'C:\dev\prod\metrikaHelper.py'
from datetime import timedelta
from datetime import datetime
import pandas as pd
import psycopg2
import numpy
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
def getDirectCallByProjectID(projectID,date1,date2):
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT id, duration, phonenumbera, date FROM calltrackinginfo where projectid={0} and campaign='direct' and date >= '{1}' and date <= '{2}'".format(projectID,date1,date2))
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCallByRow(row))
    return result
def getCallByRow(row):
    return {'id': row[0], 'duration': row[1], 'phonenumbera': row[2], 'date':row[3]}

Определяем даты контрольного и тестового периода

In [2]:
startDate = '2016-03-24'
endDate = '2016-04-12'
delta = datetime.strptime(endDate,'%Y-%m-%d')-datetime.strptime(startDate,'%Y-%m-%d')+timedelta(1)
controlStartDate = (datetime.strptime(startDate,'%Y-%m-%d')-delta).strftime('%Y-%m-%d')
controlEndDate = (datetime.strptime(startDate,'%Y-%m-%d')-timedelta(1)).strftime('%Y-%m-%d')
print controlStartDate,controlEndDate
print startDate,endDate

2016-03-04 2016-03-23
2016-03-24 2016-04-12


vtb24leasing.ru - Определяем сумму расходов по контрольному и тестовому периоду

In [3]:
controlSum = 37253.67
testSum = 31557.30

vtb24leasing.ru - Рассчитываем показатели

In [4]:
project = u'vtb24leasing.ru'
VipProjectInfo = getVipProjectByDomain(project)
print VipProjectInfo
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[0]['metrikalogin'],
                                                 'yandexPassword':VipProjectInfo[0]['metrikapassword'],
                                                 'domain':VipProjectInfo[0]['domain']})

print metrikaCredentials
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[0]['projectid'])])
otherMetrics = 'ym:s:visits'
metrics = ','.join([otherMetrics,goalsMetrics])
dimensions='ym:s:dateTime'
date1 = controlStartDate
date2 = endDate
token = metrikaCredentials['Token']
counterId = metrikaCredentials['counterID'][0]
params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset=1',
                  "filters=ym:s:UTMMedium=='direct'"])
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
print url
df = pd.read_csv(url,encoding='utf-8')
total_visits = int(df[df[u'﻿"Дата и время визита"'] == u'Итого и средние'][u'Визиты'])
current_visits = int(df[df[u'﻿"Дата и время визита"'] != u'Итого и средние'][u'Визиты'].sum())
if current_visits < total_visits:
    offset = 100001
    while current_visits < total_visits:
        params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset={0}'.format(offset),
                  "filters=ym:s:UTMMedium=='direct'"])
        url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
        extra_df = pd.read_csv(url,encoding='utf-8')
        total_df = pd.concat([df,extra_df])
        df = total_df.reset_index(drop=True)
        current_visits = int(df[df[u'﻿"Дата и время визита"'] != u'Итого и средние'][u'Визиты'].sum())
        offset += 100000
if len(df) < 500:
    print 'Not enough data'
else:
    df.rename(columns={u'﻿"Дата и время визита"': 'datetime',
                        u'Визиты': 'visits'
                       }, inplace=True)
    for i in xrange(0,len(df)):
        df.loc[i,'sumCPAVisits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df['domain'] = VipProjectInfo[0]['domain']
    df = df[~df[u'datetime'].isin([u'Итого и средние'])]
    cpaVisitsCount = df['sumCPAVisits'].sum()
print 'Общее количество визитов:',df['visits'].sum()
print 'Общее количество выполнений целей:',cpaVisitsCount
df['datetime'] = pd.to_datetime(df['datetime'])
df['hours'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.weekday
df['date'] = df['datetime'].dt.date

calls_df = pd.DataFrame(getDirectCallByProjectID(VipProjectInfo[0]['projectid'],date1,date2))
calls_df = calls_df[calls_df['duration'] > 1]
calls_df['datetime'] = pd.to_datetime(calls_df['date'])
calls_df['hours'] = calls_df['date'].dt.hour
calls_df['dayofweek'] = calls_df['date'].dt.weekday
calls_df['date'] = calls_df['datetime'].dt.date
calls_df = calls_df.drop_duplicates(subset=['phonenumbera'])
print 'Общее количество звонков:',len(calls_df)

grouped_calls = calls_df[['date','hours','dayofweek','phonenumbera']].groupby(['date','hours','dayofweek'],as_index=False).count().rename(columns={'phonenumbera':'calls'})
grouped_visits = df[['date','hours','dayofweek','sumCPAVisits','visits']].groupby(['date','hours','dayofweek'],as_index=False).sum()
merged_df = pd.merge(grouped_calls,grouped_visits,how='outer',on=['date','hours','dayofweek'])
for i in xrange(0,len(merged_df)):
    if (merged_df.loc[i,'date'] >= datetime.strptime(startDate,'%Y-%m-%d').date()) and (merged_df.loc[i,'date'] <= datetime.strptime(endDate,'%Y-%m-%d').date()):
        merged_df.loc[i,'period'] = 'Test'
    else:
        merged_df.loc[i,'period'] = 'Control'

print 'Расход по контрольной выборке:',controlSum
print 'Расход по тестовой выборке:',testSum
controlVisits = merged_df[merged_df['period'] == 'Control']['visits'].sum()
testVisits = merged_df[merged_df['period'] == 'Test']['visits'].sum()
print 'Визиты по контрольной выборке:',controlVisits
print 'Визиты по тестовой выборке:',testVisits
controlReaches = merged_df[merged_df['period'] == 'Control']['sumCPAVisits'].sum()+merged_df[merged_df['period'] == 'Control']['calls'].sum()
testReaches = merged_df[merged_df['period'] == 'Test']['sumCPAVisits'].sum()+merged_df[merged_df['period'] == 'Test']['calls'].sum()
print 'Достижений целей+звонков по контрольной выборке:',controlReaches
print 'Достижений целей+звонков по тестовой выборке:',testReaches
print 'Коэффициент конверсии в контрольной выборке:',controlReaches/controlVisits*100
print 'Коэффициент конверсии в тестовой выборке:',testReaches/testVisits*100
controlCpa = controlSum/controlReaches
testCpa = testSum/testReaches
print 'CPA в контрольной выборке',controlCpa
print 'CPA в тестовой выборке',testCpa

[{'metrikalogin': 'vtb24leasing.ru@yandex.ru', 'projectid': 2410910L, 'domain': 'vtb24leasing.ru', 'metrikapassword': 'rt8lk9v6'}]
{'Token': u'288e1aec99a34ab1be4991ac7841d14c', 'counterID': [7820509]}
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:dateTime&metrics=ym:s:visits,ym:s:goal16516325visits,ym:s:goal16516335visits,ym:s:goal16516330visits,ym:s:goal869704visits&date1=2016-03-04&date2=2016-04-12&ids=7820509&oauth_token=288e1aec99a34ab1be4991ac7841d14c&limit=100000&offset=1&filters=ym:s:UTMMedium=='direct'
Общее количество визитов: 10279.0
Общее количество выполнений целей: 83.0
Общее количество звонков: 93
Расход по контрольной выборке: 37253.67
Расход по тестовой выборке: 31557.3
Визиты по контрольной выборке: 5481.0
Визиты по тестовой выборке: 4798.0
Достижений целей+звонков по контрольной выборке: 82.0
Достижений целей+звонков по тестовой выборке: 94.0
Коэффициент конверсии в контрольной выборке: 1.49607735815
Коэффициент конверсии в тестовой выборке: 1.9

In [5]:
merged_df.to_clipboard(decimal=',')

restorate.ru - Определяем сумму расходов по контрольному и тестовому периоду

In [6]:
controlSum = 15317.98
testSum = 10382.15

In [7]:
project = u'restorate.ru'
VipProjectInfo = getVipProjectByDomain(project)
print VipProjectInfo
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[0]['metrikalogin'],
                                                 'yandexPassword':VipProjectInfo[0]['metrikapassword'],
                                                 'domain':VipProjectInfo[0]['domain']})

print metrikaCredentials
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[0]['projectid'])])
otherMetrics = 'ym:s:visits'
metrics = ','.join([otherMetrics,goalsMetrics])
dimensions='ym:s:dateTime'
date1 = controlStartDate
date2 = endDate
token = metrikaCredentials['Token']
counterId = metrikaCredentials['counterID'][0]
params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset=1',
                  "filters=ym:s:UTMMedium=='direct'"])
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
print url
df = pd.read_csv(url,encoding='utf-8')
total_visits = int(df[df[u'﻿"Дата и время визита"'] == u'Итого и средние'][u'Визиты'])
current_visits = int(df[df[u'﻿"Дата и время визита"'] != u'Итого и средние'][u'Визиты'].sum())
if current_visits < total_visits:
    offset = 100001
    while current_visits < total_visits:
        params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset={0}'.format(offset),
                  "filters=ym:s:UTMMedium=='direct'"])
        url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
        extra_df = pd.read_csv(url,encoding='utf-8')
        total_df = pd.concat([df,extra_df])
        df = total_df.reset_index(drop=True)
        current_visits = int(df[df[u'﻿"Дата и время визита"'] != u'Итого и средние'][u'Визиты'].sum())
        offset += 100000
if len(df) < 500:
    print 'Not enough data'
else:
    df.rename(columns={u'﻿"Дата и время визита"': 'datetime',
                        u'Визиты': 'visits'
                       }, inplace=True)
    for i in xrange(0,len(df)):
        df.loc[i,'sumCPAVisits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df['domain'] = VipProjectInfo[0]['domain']
    df = df[~df[u'datetime'].isin([u'Итого и средние'])]
    cpaVisitsCount = df['sumCPAVisits'].sum()
print 'Общее количество визитов:',df['visits'].sum()
print 'Общее количество выполнений целей:',cpaVisitsCount
df['datetime'] = pd.to_datetime(df['datetime'])
df['hours'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.weekday
df['date'] = df['datetime'].dt.date

calls_df = pd.DataFrame(getDirectCallByProjectID(VipProjectInfo[0]['projectid'],date1,date2))
if len(calls_df) > 0:
    calls_df = calls_df[calls_df['duration'] > 1]
    calls_df['datetime'] = pd.to_datetime(calls_df['date'])
    calls_df['hours'] = calls_df['date'].dt.hour
    calls_df['dayofweek'] = calls_df['date'].dt.weekday
    calls_df['date'] = calls_df['datetime'].dt.date
    calls_df = calls_df.drop_duplicates(subset=['phonenumbera'])
    print 'Общее количество звонков:',len(calls_df)
else:
    print 'Нет звонков'

if len(calls_df) > 0:
    grouped_calls = calls_df[['date','hours','dayofweek','phonenumbera']].groupby(['date','hours','dayofweek'],as_index=False).count().rename(columns={'phonenumbera':'calls'})
grouped_visits = df[['date','hours','dayofweek','sumCPAVisits','visits']].groupby(['date','hours','dayofweek'],as_index=False).sum()
if len(calls_df) > 0:
    merged_df = pd.merge(grouped_calls,grouped_visits,how='outer',on=['date','hours','dayofweek'])
else:
    merged_df = grouped_visits
for i in xrange(0,len(merged_df)):
    if (merged_df.loc[i,'date'] >= datetime.strptime(startDate,'%Y-%m-%d').date()) and (merged_df.loc[i,'date'] <= datetime.strptime(endDate,'%Y-%m-%d').date()):
        merged_df.loc[i,'period'] = 'Test'
    else:
        merged_df.loc[i,'period'] = 'Control'

print 'Расход по контрольной выборке:',controlSum
print 'Расход по тестовой выборке:',testSum
controlVisits = merged_df[merged_df['period'] == 'Control']['visits'].sum()
testVisits = merged_df[merged_df['period'] == 'Test']['visits'].sum()
print 'Визиты по контрольной выборке:',controlVisits
print 'Визиты по тестовой выборке:',testVisits
if len(calls_df) > 0:
    controlReaches = merged_df[merged_df['period'] == 'Control']['sumCPAVisits'].sum()+merged_df[merged_df['period'] == 'Control']['calls'].sum()
    testReaches = merged_df[merged_df['period'] == 'Test']['sumCPAVisits'].sum()+merged_df[merged_df['period'] == 'Test']['calls'].sum()
else:
    controlReaches = merged_df[merged_df['period'] == 'Control']['sumCPAVisits'].sum()
    testReaches = merged_df[merged_df['period'] == 'Test']['sumCPAVisits'].sum()
print 'Достижений целей+звонков по контрольной выборке:',controlReaches
print 'Достижений целей+звонков по тестовой выборке:',testReaches
print 'Коэффициент конверсии в контрольной выборке:',controlReaches/controlVisits*100
print 'Коэффициент конверсии в тестовой выборке:',testReaches/testVisits*100
controlCpa = controlSum/controlReaches
testCpa = testSum/testReaches
print 'CPA в контрольной выборке',controlCpa
print 'CPA в тестовой выборке',testCpa

[{'metrikalogin': 'RookeeXML@yandex.ru', 'projectid': 2406006L, 'domain': 'restorate.ru', 'metrikapassword': 'pass-RookeeXML'}]
{'Token': u'71e4f97915d34895b4de3b45dbb6beef', 'counterID': [23008126]}
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:dateTime&metrics=ym:s:visits,ym:s:goal16355695visits&date1=2016-03-04&date2=2016-04-12&ids=23008126&oauth_token=71e4f97915d34895b4de3b45dbb6beef&limit=100000&offset=1&filters=ym:s:UTMMedium=='direct'
Общее количество визитов: 1738.0
Общее количество выполнений целей: 75.0
Нет звонков
Расход по контрольной выборке: 15317.98
Расход по тестовой выборке: 10382.15
Визиты по контрольной выборке: 1109.0
Визиты по тестовой выборке: 629.0
Достижений целей+звонков по контрольной выборке: 48.0
Достижений целей+звонков по тестовой выборке: 27.0
Коэффициент конверсии в контрольной выборке: 4.32822362489
Коэффициент конверсии в тестовой выборке: 4.29252782194
CPA в контрольной выборке 319.124583333
CPA в тестовой выборке 384.524074074


In [8]:
merged_df.to_clipboard(decimal=',')